In [2]:
import cv2

In [3]:
# Charger la vidéo d'origine
input_video_path = "test_videos/football_video.mp4"  # Remplace par le chemin de ta vidéo
output_video_path = "test_videos/match_football_10fps.mp4"

# Ouvrir la vidéo
cap = cv2.VideoCapture(input_video_path)

# Vérifier si la vidéo est bien chargée
if not cap.isOpened():
    print("Erreur lors du chargement de la vidéo.")
    exit()

In [4]:
# Obtenir les propriétés de la vidéo
original_fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (frame_width, frame_height)

# Définir le codec et initialiser l'enregistreur vidéo
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec pour le format MP4
out = cv2.VideoWriter(output_video_path, fourcc, 10, frame_size)

# Lire et écrire les frames à 10 FPS
frame_interval = original_fps // 10  # Intervalle pour sélectionner les frames

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Écrire uniquement les frames à l'intervalle défini
    if frame_count % frame_interval == 0:
        out.write(frame)

    frame_count += 1

# Libérer les ressources
cap.release()
out.release()

print(f"Vidéo convertie à 10 FPS sauvegardée sous : {output_video_path}")


Vidéo convertie à 10 FPS sauvegardée sous : test_videos/match_football_10fps.mp4


### Now that we have the video in 10 FPS for less computation power needs, let us start the prediction

In [10]:
import cv2
from PIL import Image, ImageDraw, ImageFont
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
import numpy as np

In [6]:
# Charger la vidéo
input_video_path = "test_videos/match_football_10fps.mp4"
cap = cv2.VideoCapture(input_video_path)

# Vérifier si la vidéo est bien chargée
if not cap.isOpened():
    print("Erreur lors du chargement de la vidéo.")
    exit()

# Lire la première frame
ret, frame = cap.read() # Lis uniquement la première frame, ret est un booléen
if ret:
    # Convertir la frame (OpenCV -> PIL)
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image.show()  # Affiche la première frame comme une image
else:
    print("Erreur lors de la lecture de la première frame.")

# Libérer les ressources pour l'instant
cap.release()


In [7]:
# Charger le modèle et le processeur
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

# Préparer l'image pour le modèle
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# Convertir les outputs en format COCO
target_sizes = torch.tensor([image.size[::-1]])  # Taille de l'image (H, W)
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

# Annoter l'image avec PIL
draw = ImageDraw.Draw(image)

# Optionnel : Charger une police pour le texte
try:
    font = ImageFont.truetype("arial.ttf", size=15)
except IOError:
    font = ImageFont.load_default()

# Dessiner les boîtes et les labels
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    x_min, y_min, x_max, y_max = map(int, box)

    # Dessiner le rectangle
    draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)

    # Ajouter le label et le score
    label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 2)}"
    text_bbox = draw.textbbox((0, 0), label_text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    text_bg = [x_min, y_min - text_height, x_min + text_width, y_min]

    draw.rectangle(text_bg, fill="red")
    draw.text((x_min, y_min - text_height), label_text, fill="white", font=font)

# Afficher l'image annotée
image.show()

#### It works with one frame so let's do it for the entire video

In [11]:
# Réinitialiser le lecteur de vidéo
cap = cv2.VideoCapture(input_video_path)

# Créer une liste pour stocker les frames annotées
annotated_frames = []

frame_count = 0  # Compteur pour suivre le nombre de frames traitées

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Fin de la vidéo

    # Convertir la frame OpenCV en PIL.Image
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Passer l'image au modèle
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Post-traitement des résultats
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    # Annoter l'image
    draw = ImageDraw.Draw(image)

    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        x_min, y_min, x_max, y_max = map(int, box)
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)
        label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 2)}"
        text_bbox = draw.textbbox((0, 0), label_text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_bg = [x_min, y_min - text_height, x_min + text_width, y_min]
        draw.rectangle(text_bg, fill="red")
        draw.text((x_min, y_min - text_height), label_text, fill="white", font=font)

    # Ajouter la frame annotée à la liste (convertir PIL -> OpenCV)
    annotated_frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    annotated_frames.append(annotated_frame)

    frame_count += 1
    if frame_count % 10 == 0:
        print(f"{frame_count} frames traitées...")

cap.release()
print("Toutes les frames ont été annotées.")


10 frames traitées...
20 frames traitées...
30 frames traitées...
40 frames traitées...
50 frames traitées...
60 frames traitées...
70 frames traitées...
80 frames traitées...
90 frames traitées...
100 frames traitées...
110 frames traitées...
120 frames traitées...
130 frames traitées...
140 frames traitées...
150 frames traitées...
Toutes les frames ont été annotées.


Now we have to save all the frames in one video:

In [12]:
# Spécifiez le chemin de sortie pour la vidéo annotée
output_video_path = "results/videos/annotated_football_video.mp4"

# Définir les propriétés de la vidéo de sortie (format, FPS, dimensions)
frame_height, frame_width, _ = annotated_frames[0].shape  # Taille de la première frame
fps = 10  # Choisir 10 FPS comme spécifié

# Initialiser le VideoWriter
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec pour MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Ajouter chaque frame annotée à la vidéo
for frame in annotated_frames:
    out.write(frame)

# Libérer les ressources
out.release()

print(f"La vidéo annotée a été sauvegardée sous : {output_video_path}")


La vidéo annotée a été sauvegardée sous : annotated_football_video.mp4
